In [2]:
import csv

# Rutas para guardar las imágenes y etiquetas
image_paths = []
labels = []

# Cargar y procesar el archivo CSV
with open("../test/tempImagesLabels.csv", "r") as file:
    reader = csv.reader(file)
    next(reader)  # Saltar la cabecera

    for row in reader:
        # Obtener ruta de la imagen y convertir las etiquetas de string a lista de enteros
        image_paths.append(row[0])
        labels.append(list(map(int, row[1].strip("[]").split(", "))))

# Verificar resultados
print("Imágenes:", image_paths[:3])  # Muestra los primeros 3 elementos
print("Etiquetas:", labels[:3])      # Muestra los primeros 3 elementos

Imágenes: ['tempImages/002_24fab375.jpg', 'tempImages/010_18d7b27c.jpg', 'tempImages/010_2f9c83bc.jpg']
Etiquetas: [[1, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0]]


In [3]:
import numpy as np
import tensorflow as tf

# Dimensiones a las que redimensionar las imágenes
IMG_HEIGHT = 128
IMG_WIDTH = 128

# Cargar imágenes y etiquetas
image_data = []
for image_path in image_paths:
    # Cargar imagen desde archivo
    img = tf.keras.utils.load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    # Convertir la imagen a un array numpy
    img_array = tf.keras.utils.img_to_array(img)
    # Normalizar los valores de los píxeles (0-255 a 0-1)
    img_array = img_array / 255.0
    image_data.append(img_array)

# Convertir a tensores para entrenar
image_data = np.array(image_data)
labels = np.array(labels)

# Verificar formas
print("Forma de los datos de imágenes:", image_data.shape)
print("Forma de las etiquetas:", labels.shape)

Forma de los datos de imágenes: (10, 128, 128, 3)
Forma de las etiquetas: (10, 6)


In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
# Dividir datos en entrenamiento (70%), validación (15%) y prueba (15%)
X_train, X_temp, y_train, y_temp = train_test_split(image_data, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Datos de entrenamiento: {X_train.shape}")
print(f"Datos de validación: {X_val.shape}")
print(f"Datos de prueba: {X_test.shape}")

Datos de entrenamiento: (7, 128, 128, 3)
Datos de validación: (1, 128, 128, 3)
Datos de prueba: (2, 128, 128, 3)


In [11]:
# Inicializar el generador de datos para entrenamiento con data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalizar las imágenes
    rotation_range=40,  # Rotar las imágenes aleatoriamente
    width_shift_range=0.2,  # Desplazamiento horizontal
    height_shift_range=0.2,  # Desplazamiento vertical
    shear_range=0.2,  # Cizallamiento
    zoom_range=0.2,  # Zoom aleatorio
    horizontal_flip=True,  # Voltear imágenes horizontalmente
    fill_mode='nearest'  # Método de relleno para píxeles faltantes
)

# Generador de datos para la validación (sin data augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

# Ajustar las imágenes a los generadores
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

In [12]:
print("Total samples in train_generator:", len(X_train))

Total samples in train_generator: 7


In [12]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [13]:
# Definir la función de modelo con parámetros de búsqueda
def build_model(hp):
    model = Sequential()

    # Primera capa Conv2D con parámetros de búsqueda para el número de filtros y tamaño del kernel
    model.add(Conv2D(
        filters=hp.Int('filters_1', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size_1', values=[3, 5]),
        activation='relu',
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Segunda capa Conv2D con parámetros de búsqueda
    model.add(Conv2D(
        filters=hp.Int('filters_2', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size_2', values=[3, 5]),
        activation='relu'
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Aplanar la salida
    model.add(Flatten())

    # Capa densa con número de unidades a ser optimizado
    model.add(Dense(
        hp.Int('units', min_value=64, max_value=256, step=64),
        activation='relu'
    ))

    # Capa de salida con activación sigmoid para clasificación binaria
    model.add(Dense(labels.shape[1], activation='sigmoid'))

    # Compilar el modelo con tasa de aprendizaje como un parámetro de búsqueda
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
# Definir el objeto de búsqueda de hiperparámetros
tuner = kt.Hyperband(
    build_model,  # Función que construye el modelo
    objective='val_accuracy',  # Objetivo de optimización
    max_epochs=10,  # Número máximo de épocas
    hyperband_iterations=2,  # Número de iteraciones de Hyperband
    directory='cnn_models',  # Directorio donde se guardarán los resultados
    project_name='cnn_multi_label'  # Nombre del proyecto
)

In [15]:
# Realizar la búsqueda de hiperparámetros
tuner.search(train_generator, epochs=10, validation_data=val_generator)

Trial 57 Complete [00h 00m 13s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 04m 47s


In [16]:
# Obtener el mejor modelo
best_model = tuner.get_best_models(num_models=1)[0]

In [17]:
from sklearn.metrics import f1_score, classification_report

THRESHOLD = 0.8

# Predecir en el conjunto de prueba
y_pred = best_model.predict(X_test)
y_pred_binary = (y_pred > THRESHOLD).astype(int)  # Convertir probabilidades a etiquetas binarias

# Calcular F1 Score
f1 = f1_score(y_test, y_pred_binary, average='weighted')  # Para clasificación multi-clase
print(f"F1 Score (ponderado): {f1}")

# Reporte de clasificación detallado
print("\nReporte de clasificación:\n")
print(classification_report(y_test, y_pred_binary))

1/1 [==============================] - 0s 88ms/step
F1 Score (ponderado): 0.6666666666666666

Reporte de clasificación:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.50      1.00      0.67         1
           5       0.00      0.00      0.00         0

   micro avg       0.12      1.00      0.22         1
   macro avg       0.08      0.17      0.11         1
weighted avg       0.50      1.00      0.67         1
 samples avg       0.12      0.50      0.20         1



c:\Users\gonibix23\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\gonibix23\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\gonibix23\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_

In [18]:
epochs = range(1, len(history.history['accuracy']) + 1)

# Pérdida (Loss)
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'r', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Precisión (Accuracy)
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'r', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

NameError: name 'history' is not defined